In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.4)
import matplotlib.pyplot as plt
%matplotlib inline

from itertools import combinations
import math
import statistics

from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import chi2
from scipy.stats import poisson

import time

from datetime import datetime
import matplotlib.dates as mdates
import plotly.express as px
import lightgbm as lgbm
from termcolor import colored
import itertools
import warnings
warnings.filterwarnings("ignore")

import sklearn
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, TimeSeriesSplit, GroupKFold, cross_validate
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture

# UMAP
import umap
# import umap.plot

# Models
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import early_stopping
from lightgbm import LGBMClassifier, LGBMRegressor
import xgboost as xgb

from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMClassifier
from lightgbm.sklearn import LGBMRegressor

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

# sklearn에서 배웠던 분류 모델들을 불러와봅니다.
from sklearn.linear_model import SGDClassifier      # 1. Linear Classifier
from sklearn.linear_model import LogisticRegression # 2. Logistic Regression
from sklearn.tree import DecisionTreeClassifier     # 3. Decision Tree
from sklearn.ensemble import RandomForestClassifier # 4. Random Forest

# 평가 지표
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

Duplicate key in file WindowsPath('C:/Users/Hyoungrak Oh/.conda/envs/ETA3.9/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 257 ('font.family: sans-serif')
Duplicate key in file WindowsPath('C:/Users/Hyoungrak Oh/.conda/envs/ETA3.9/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 406 ('axes.unicode_minus: True  # use Unicode for the minus symbol rather than hyphen.  See')
C:\Users\Hyoungrak Oh\.conda\envs\ETA3.9\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
rawtrain = pd.read_csv('./data/train.csv')
rawtest = pd.read_csv('./data/test.csv')
rawtarget = rawtrain['failure']
rawcolumns = rawtrain.columns
rawdata = pd.concat([rawtrain, rawtest], ignore_index = True)

In [3]:
data = pd.concat([rawtrain, rawtest], ignore_index = True)
data

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0.0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0.0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0.0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0.0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47340,47340,I,144.74,material_7,material_5,9,5,0,4,9,...,11.691,NaN,19.771,11.562,17.246,15.131,15.209,NaN,696.466,NaN
47341,47341,I,74.53,material_7,material_5,9,5,4,8,7,...,10.980,16.027,15.694,13.564,15.494,15.296,13.812,16.501,613.249,NaN
47342,47342,I,67.73,material_7,material_5,9,5,10,11,2,...,10.436,16.137,20.612,11.134,16.519,15.525,14.175,17.728,783.349,NaN
47343,47343,I,126.15,material_7,material_5,9,5,8,16,11,...,12.672,15.422,19.496,9.319,15.817,17.403,16.437,15.179,745.210,NaN


In [4]:
X = pd.read_csv('./data/X.csv')
y = pd.read_csv('./data/y.csv')
test_df = pd.read_csv('./data/test_df.csv')

In [5]:
X

,loading,attribute_2,attribute_3,material_7material_8,material_5material_5,material_7material_5,material_7material_6,material_5material_6,material_7material_7,measurement_0,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,80.10,9,5,1,0,0,0,0,0,7.0,...,20.155,10.6720,15.859,17.5940,15.193,15.029,17.9985,13.034,14.6840,764.100
1,84.89,9,5,1,0,0,0,0,0,14.0,...,17.889,12.4480,17.947,17.9150,11.755,14.732,15.4250,14.395,15.6310,682.057
2,82.43,9,5,1,0,0,0,0,0,12.0,...,18.288,12.7150,15.607,20.6745,13.798,16.711,18.6310,14.094,17.9460,663.376
3,101.07,9,5,1,0,0,0,0,0,13.0,...,19.060,12.4710,16.346,18.3770,10.020,15.250,15.5620,16.154,17.1720,826.282
4,188.06,9,5,1,0,0,0,0,0,9.0,...,18.093,10.3370,17.082,19.9320,12.428,16.182,12.7600,13.153,16.4120,579.885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25334,158.95,6,9,0,0,0,1,0,0,6.0,...,19.354,11.0055,12.177,17.9420,10.112,15.795,18.5720,16.144,16.3615,729.131
25335,146.02,6,9,0,0,0,1,0,0,10.0,...,19.563,11.2420,14.179,20.5640,10.234,14.450,14.3220,13.146,16.4710,853.924
25336,115.62,6,9,0,0,0,1,0,0,1.0,...,19.279,11.4070,16.437,17.4760,8.668,15.069,16.5990,15.590,14.0650,750.364
25337,106.38,6,9,0,0,0,1,0,0,2.0,...,19.358,11.3920,17.064,17.8140,14.928,16.273,15.4850,13.624,12.8650,730.156


In [6]:
y['failure'] = y['failure'].astype(int)
y

,failure
0,0
1,0
2,0
3,0
4,0
...,...
25334,0
25335,0
25336,0
25337,0


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 42)

In [8]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from sklearn.metrics import mean_absolute_error

In [9]:
# def objectiveXGB(trial: Trial, X, y, test):
#     param = {
#         "n_estimators" : trial.suggest_int('n_estimators', 500, 4000),
#         'max_depth':trial.suggest_int('max_depth', 8, 16),
#         'min_child_weight':trial.suggest_int('min_child_weight', 1, 300),
#         'gamma':trial.suggest_int('gamma', 1, 3),
#         'learning_rate': 0.01,
#         'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.1),
#         'nthread' : -1,
# #         'tree_method': 'gpu_hist',
# #         'predictor': 'gpu_predictor',
#         'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
#         'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
#         'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
#         'random_state': 42
#     }
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 42)
    
# #     y_train = y_train.reshape(-1, 1)
# #     y_test  = y_test.reshape(-1, 1)

#     model = xgb.XGBRegressor(**param)
#     xgb_model = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
#     score = roc_auc_score(xgb_model.predict(X_test), y_test)

#     return score

In [10]:
# study = optuna.create_study(direction='minimize',sampler=TPESampler())
# study.optimize(lambda trial : objectiveXGB(trial, X,  y, X_test), n_trials=50)
# print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

# best_param = study.best_trial.params
# xgbReg = train(xgb.XGBRegressor(**best_param, random_state=42, learning_rate=0.01, nthread=-1))

[I 2022-08-19 16:38:43,831] A new study created in memory with name: no-name-3fb69b66-1a14-4e4e-b8e1-5ecbf84f58f7
[W 2022-08-19 16:39:55,681] Trial 0 failed because of the following error: ValueError('continuous format is not supported')
Traceback (most recent call last):
  File "C:\Users\Hyoungrak Oh\.conda\envs\ETA3.9\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Hyoungrak Oh\AppData\Local\Temp\ipykernel_11968\3754533895.py", line 2, in <lambda>
    study.optimize(lambda trial : objectiveXGB(trial, X,  y, X_test), n_trials=50)
  File "C:\Users\Hyoungrak Oh\AppData\Local\Temp\ipykernel_11968\2880409150.py", line 24, in objectiveXGB
    score = roc_auc_score(xgb_model.predict(X_test), y_test)
  File "C:\Users\Hyoungrak Oh\.conda\envs\ETA3.9\lib\site-packages\sklearn\metrics\_ranking.py", line 577, in roc_auc_score
    return _average_binary_score(
  File "C:\Users\Hyoungrak Oh\.conda\envs\ETA3.9\lib\site-package

ValueError: continuous format is not supported

In [ ]:
xgb.__version__

In [11]:
y

,failure
0,0
1,0
2,0
3,0
4,0
...,...
25334,0
25335,0
25336,0
25337,0


In [ ]:
y = y['failure'].astype(int)

In [11]:
X

,loading,attribute_2,attribute_3,material_7material_8,material_5material_5,material_7material_5,material_7material_6,material_5material_6,material_7material_7,measurement_0,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,80.10,9,5,1,0,0,0,0,0,7.0,...,20.155,10.6720,15.859,17.5940,15.193,15.029,17.9985,13.034,14.6840,764.100
1,84.89,9,5,1,0,0,0,0,0,14.0,...,17.889,12.4480,17.947,17.9150,11.755,14.732,15.4250,14.395,15.6310,682.057
2,82.43,9,5,1,0,0,0,0,0,12.0,...,18.288,12.7150,15.607,20.6745,13.798,16.711,18.6310,14.094,17.9460,663.376
3,101.07,9,5,1,0,0,0,0,0,13.0,...,19.060,12.4710,16.346,18.3770,10.020,15.250,15.5620,16.154,17.1720,826.282
4,188.06,9,5,1,0,0,0,0,0,9.0,...,18.093,10.3370,17.082,19.9320,12.428,16.182,12.7600,13.153,16.4120,579.885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25334,158.95,6,9,0,0,0,1,0,0,6.0,...,19.354,11.0055,12.177,17.9420,10.112,15.795,18.5720,16.144,16.3615,729.131
25335,146.02,6,9,0,0,0,1,0,0,10.0,...,19.563,11.2420,14.179,20.5640,10.234,14.450,14.3220,13.146,16.4710,853.924
25336,115.62,6,9,0,0,0,1,0,0,1.0,...,19.279,11.4070,16.437,17.4760,8.668,15.069,16.5990,15.590,14.0650,750.364
25337,106.38,6,9,0,0,0,1,0,0,2.0,...,19.358,11.3920,17.064,17.8140,14.928,16.273,15.4850,13.624,12.8650,730.156


In [13]:
y

,failure
0,0
1,0
2,0
3,0
4,0
...,...
25334,0
25335,0
25336,0
25337,0


In [15]:
test = test_df
test

,loading,attribute_2,attribute_3,material_7material_8,material_5material_5,material_7material_5,material_7material_6,material_5material_6,material_7material_7,measurement_0,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,119.57,6,4,0,0,0,0,1,0,6.0,...,18.654,10.802,15.9090,18.070,13.772,13.659,16.825,13.742,17.710,634.612
1,113.51,6,4,0,0,0,0,1,0,11.0,...,19.368,12.032,13.9980,17.030,12.473,17.468,16.708,14.776,14.102,537.037
2,112.16,6,4,0,0,0,0,1,0,8.0,...,17.774,11.743,17.0460,18.086,10.907,13.363,15.737,17.065,16.021,658.995
3,112.72,6,4,0,0,0,0,1,0,8.0,...,18.948,11.790,18.1650,16.163,10.933,15.501,15.667,12.620,16.111,594.301
4,208.00,6,4,0,0,0,0,1,0,14.0,...,19.141,12.370,14.5780,17.849,11.941,16.070,16.183,13.324,17.150,801.044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20770,144.74,9,5,0,0,1,0,0,0,0.0,...,18.573,11.691,14.7005,19.771,11.562,17.246,15.131,15.209,16.994,696.466
20771,74.53,9,5,0,0,1,0,0,0,4.0,...,17.917,10.980,16.0270,15.694,13.564,15.494,15.296,13.812,16.501,613.249
20772,67.73,9,5,0,0,1,0,0,0,10.0,...,19.630,10.436,16.1370,20.612,11.134,16.519,15.525,14.175,17.728,783.349
20773,126.15,9,5,0,0,1,0,0,0,8.0,...,19.575,12.672,15.4220,19.496,9.319,15.817,17.403,16.437,15.179,745.210


In [16]:
def objective(trial: Trial, X, y, test):
    param = {
        "n_estimators" : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 300),
        'gamma':trial.suggest_int('gamma', 1, 3),
        'learning_rate': 0.01,
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.1),
        'nthread' : -1,
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 42)
    model = xgb.XGBRegressor(**param)
    xgb_model = model.fit(X_train, y_train)
    score = roc_auc_score(xgb_model.predict(X_test), y_test)
    return score

In [17]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2022-08-19 17:44:10,619] A new study created in memory with name: no-name-9c4a33a1-543e-422c-b40f-8f4be4f30fd8
[W 2022-08-19 17:44:10,623] Trial 0 failed because of the following error: TypeError("objective() missing 3 required positional arguments: 'X', 'y', and 'test'")
Traceback (most recent call last):
  File "C:\Users\Hyoungrak Oh\.conda\envs\ETA3.9\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
TypeError: objective() missing 3 required positional arguments: 'X', 'y', and 'test'


TypeError: objective() missing 3 required positional arguments: 'X', 'y', and 'test'